In [11]:
import pandas as pd
from keras_wc_embd import get_dicts_generator
import keras
from keras_wc_embd import get_embedding_layer
from keras_wc_embd import WordCharEmbd
import numpy as np

In [12]:
train_data_path = "./cleaned_data/sample_train_task1.csv"
valid_data_path = './cleaned_data/sample_valid_task1.csv'
test_data_path = "./cleaned_data/cleaned_test_data_for_subtask1.csv"

In [13]:
train_data = pd.read_csv(train_data_path)
valid_data = pd.read_csv(valid_data_path)
test_data = pd.read_csv(test_data_path)

In [14]:
train_data.head()

,#1_tweetid,#2_tweet,#3_country_label,#2_tweet_clean_V0,#2_tweet_clean_farasaV0,#2_tweet_clean_V1,#2_tweet_clean_V2,#2_tweet_clean_V3,#classes_id,#len_of_2_tweet_clean_V0,cluster_on_length
0,TRAIN_17392,الحب ما بينشرى ولا بينباع...,Lebanon,الحب ما بينشرى ولا بينباع,ال حب ما ب ينشرى و لا ب ينباع,الحب ما بينشرى ولا بينباع,الحب بينشرى بينباع,الحب بينشرى بينباع,8,5,2
1,TRAIN_15280,يعني كل يوم الصبح اروح عالرئاسه اصبح على دولته...,Jordan,يعني كل يوم الصبح اروح عالرئاسه اصبح على دولته...,يعني كل يوم ال صبح اروح عالرئاسه أصبح على دول ...,يعني كل يوم الصبح اروح عالراسه اصبح على دولته ...,يعني الصبح اروح عالراسه دولته واخذ مصروفي قروش...,يعني الصبح اروح عالراسه دولته واخذ مصروفي قروش,18,15,2
2,TRAIN_7204,تعبت وانا اداوم س١٠,Oman,تعبت وانا اداوم س١٠,تعب ت و أنا اداوم س 10,تعبت وانا اداوم,تعبت وانا اداوم,تعبت وانا اداوم,6,4,0
3,TRAIN_15564,امبارح تلاقينا قعدنا ع حجر برد و حوالينا عريان...,Egypt,امبارح تلاقينا قعدنا ع حجر برد و حوالينا عريان...,امبارح تلاقي نا قعد نا ع حجر برد و حوالي نا عر...,امبارح تلاقينا قعدنا حجر برد حوالينا عريان الش...,امبارح تلاقينا قعدنا حجر برد حوالينا عريان الش...,امبارح تلاقينا قعدنا حجر برد حوالينا عريان الش...,0,21,1
4,TRAIN_15019,النفسية محتاجة شهر في سويسرا URL,Algeria,النفسية محتاجة شهر في سويسرا,ال نفسي ة محتاج ة شهر في سويسرا,النفسيه محتاجه شهر في سويسرا,النفسيه محتاجه شهر سويسرا,النفسيه محتاجه شهر سويسرا,4,8,1


In [15]:
# dict_generator = get_dicts_generator(
#     word_min_freq=2,
#     char_min_freq=2,
#     word_ignore_case=False,
#     char_ignore_case=False,
# )

In [16]:
# for tweet in train_data['#2_tweet_clean_V1'].tolist():
#     dict_generator(tweet.split(' '))

# for tweet in test_data['#2_tweet_clean_V1'].tolist():
#     dict_generator(tweet.split(' '))

# for tweet in valid_data['#2_tweet_clean_V1'].tolist():
#     dict_generator(tweet.split(' '))

In [17]:
# word_dict, char_dict, max_word_len = dict_generator(return_dict=True)

In [18]:
# inputs, embd_layer = get_embedding_layer(
#     word_dict_len=len(word_dict),
#     char_dict_len=len(char_dict),
#     max_word_len=max_word_len,
#     word_embd_dim=300,
#     char_embd_dim=50,
#     char_hidden_dim=150,
#     char_hidden_layer_type='lstm',
# )
# model = keras.models.Model(inputs=inputs, outputs=embd_layer)
# model.summary()

In [19]:
training_tweets =  train_data['#2_tweet_clean_V1'].str.split(' ').tolist()
validation_tweets =  valid_data['#2_tweet_clean_V1'].str.split(' ').tolist()
test_tweets =  test_data['#2_tweet_clean_V1'].str.split(' ').tolist()


In [20]:
wc_embd = WordCharEmbd(
    word_min_freq=0,
    char_min_freq=0,
    word_ignore_case=False,
    char_ignore_case=False,
)

for tweet in training_tweets:
    wc_embd.update_dicts(tweet)
    
for tweet in test_tweets:
    wc_embd.update_dicts(tweet)
    
for tweet in validation_tweets:
    wc_embd.update_dicts(tweet)

In [ ]:
inputs, embd_layer = wc_embd.get_embedding_layer()
lstm_layer = keras.layers.LSTM(units=300, name='LSTM')(embd_layer)
softmax_layer = keras.layers.Dense(units=21, activation='softmax', name='Softmax')(lstm_layer)
model = keras.models.Model(inputs=inputs, outputs=softmax_layer)

model.compile(
    optimizer='adam',
    loss=keras.losses.sparse_categorical_crossentropy,
    metrics=[keras.metrics.sparse_categorical_accuracy],
)
model.summary()


def batch_generator():
    while True:
        yield wc_embd.get_batch_input(training_tweets), np.asarray([0, 1])

model.fit_generator(
    generator=batch_generator(),
    steps_per_epoch=1000,
    epochs=1,
)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Char (InputLayer)         [(None, None, 33)]   0                                            
__________________________________________________________________________________________________
Input_Word (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding_Char_Pre (Embedding)  (None, None, 33, 30) 4050        Input_Char[0][0]                 
__________________________________________________________________________________________________
Embedding_Word (Embedding)      (None, None, 300)    17067300    Input_Word[0][0]                 
______________________________________________________________________________________________

/home/omar/anaconda3/envs/NLP/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
